In [1]:
#import time
#import numpy as np
#import matplotlib.pyplot as plt
#from IPython import display

#%run "scripts/k2400.ipynb"
#%run "scripts/srs.ipynb"
#%run "scripts/fileio.ipynb"
max_n_filling= 4.5e16 #per meter^2, from max_theta=1.4deg
Tp = np.transpose
VB_over_VT = 6.545
# Exp_log is a data structure containing all data collected in a given execution of Vg_to_n.
class Vg_to_n_data:
    def __init__(self, dc_voltage,delay,IMag):
        self.dc_voltage = dc_voltage # array of dc voltages
        self.ac_voltage = np.zeros((len(dc_voltage),2)) # array of ac voltage readings (X,Y) from lockin
        self.density = np.zeros(len(dc_voltage)) # 
        self.delay = delay # acquisition delay 
        self.IMag=IMag # hall effect constant B

# lockin_iv is a function that sequentially sets a keithley to a specified array of voltages. at each
# voltage, the dc current on the keithley and the ac voltage and phase read by the lockin are recorded.
# at the end of execution, the data will be saved to an IV object (defined above). this IV object will
# also be written to a file for later retrieval.

# input parameters:
#    k2400 is a keithley object, assumed initialized
#    srs is a srs lockin object, assumed initialized
#    dc_voltage is an array of dc voltages for the keithley
#    delay is the time to delay before acquiring data at each dc voltage


# Split the total voltage difference to both top and bottom
def SweepBothGate(k2400,E3640A,V):
    sweep2400(k2400,V/2*VB_over_VT)
    if E3640A_is_flipped:
        sweepE3640A(E3640A,-V/2)
    else:
        sweepE3640A(E3640A,V/2)
def Vxy_to_n(Vxy,IMag):

    return -IMag_to_B(IMag)/consts.e/V_to_R(Vxy)
    
def filling_n_to_theta(n):
    return np.sqrt(n*np.sqrt(3)/8)*0.246e-9/(2*np.Pi)*360

# input parameters:
#    k2400 is a keithley object, assumed initialized
#    srs is a srs lockin object, assumed initialized
#    dc_voltage is an array of dc voltages for the keithley
#    delay is the time to delay before acquiring data at each dc voltage
def B_Vg_R_to_Vg_to_n(data,G_min,G_max):
    data.density = np.zeros(len(data.GateV))
    B =np.zeros(len(data.IMag))
    Rxy =np.zeros(len(data.IMag))

    for i in range(len(data.GateV)):
        for j in range(len(data.IMag)):
            B[j] = IMag_to_B(data.IMag[j])
            Rxy[j]= V_to_R(data.Vxy[j][i][0])
        a, b, r_value, p_value, std_err = stats.linregress(Rxy,B)    
        #plt.plot(x, a*x + b, 'r', label='Fitted line')
        # formula
        data.density[i]=-a/consts.e/10000 
        
    a, b, r_value, p_value, std_err = stats.linregress(data.GateV[G_min:G_max],data.density[G_min:G_max])
    
    
    fig = plt.figure(figsize=(8, 8))
    full_filename = makefilename()
    fig.suptitle(full_filename)
    #ax1=plt.subplot(1,2,1)
    plt.xlabel('Gate (V)')
    plt.ylabel(r'$n_H$ (1/cm^2)')
    plt.plot(data.GateV,data.density)
    n=len(data.GateV[G_min:G_max])
    a_std = std_err/np.sqrt(np.var(data.GateV[G_min:G_max])*n)
    b_std = std_err*np.sqrt((1/n+np.average(data.GateV[G_min:G_max])**2/np.var(data.GateV[G_min:G_max])/n))
    #ax1.lines[0].set_xdata(data.GateV)
    #ax1.lines[0].set_ydata(data.density)#plot the X component,  
    plt.plot(np.array(data.GateV), a*np.array(data.GateV) + b, 'r', label='Fitted line')
    plt.text(5,-0.5e12,r'n = (%.2e $\pm$ %.0e )V+ (%.2e $\pm$ %.0e )'%(a,a_std,b,b_std))
    return a,b
        
def Vg_to_n(k2400,srs_xy,hp6031_U,hp6031_D,GateV,delay,IMag,CutOff):
#def Vg_to_n(k2400,E3640A,srs_xy,hp6031_U,GateV,delay,IMag):


    %matplotlib notebook
    
    # create new IV object to store data in
    data = Vg_to_n_data(GateV,delay,IMag)
    
    # make a filename to save the data, for later use
    full_filename = makefilename()

    # define a figure with two panels to show the data being acquired
    fig = plt.figure(figsize=(8, 3)) 
    
    fig.suptitle(full_filename)
    ax1=plt.subplot(1,2,1)
    ax1.set_xlabel('dc voltage (V)')
    ax1.set_ylabel('number density (1/m^2)')
    plt.plot(data.dc_voltage,data.density)
    

    #turn on B
    #IsweepMag(hp6031_U,IMag) # one machine
    IsweepBoth(hp6031_U,hp6031_D,IMag) #two machine
    
    try:
        # the for loop below is the main acquisition loop. the reason the for loop lives in a try-except
        # is that if the user cancels while the sweep is running, which throws an exception, the program 
        # catches it and continues to execute below the except statement. this lets the program shut down
        # nicely and save what data has already been collected.
        print('trying')
        # main loop: iterate over dc voltages
        for i in range(len(data.dc_voltage)):
            if i>0 and abs(data.density[i-1]-data.density[0])>CutOff*max_n_filling: #not applying this voltage
                print('CutOff density reached at %1.6f'%data.dc_voltage[i-1])
            else:     
                # set voltage for both gates
                #SweepBothGate(k2400,E3640A,data.dc_voltage[i])
                print('else')
                sweep2400(k2400,data.dc_voltage[i])
                print(data.dc_voltage[i])
                #srs_xy.write('AGAN')
                #time.sleep(data.delay)
                #srs_xy.write('ARSV') # assuming the SNR is fairly stable
                #time.sleep(data.delay/2)


                print('srs')
                # wait acquisition delay 

                # read data and store it
                data.ac_voltage[i]=smart_auto_delay_read(srs_xy,delay) 
                
                data.density[i] = Vxy_to_n(data.ac_voltage[i][0],IMag)
                np.savetxt(full_filename+'.txt',data.dc_voltage[i],newline='\r\n')
                np.savetxt(full_filename+'.txt',data.ac_voltage[i],newline='\r\n')
                np.savetxt(full_filename+'.txt',data.density[i],newline='\r\n')
                print('readsrs')            
                # update plots
                ax1.lines[0].set_xdata(data.dc_voltage[0:(i+1)])
                ax1.lines[0].set_ydata(data.density[0:(i+1)])#plot the X component,
                
                a, b, r_value, p_value, std_err = stats.linregress(data.dc_voltage,data.density)    
                plt.plot(x, a**x + b, 'r', label='Fitted line')
                
                ax1.relim()
                ax1.autoscale()
                print('plot')
                #ax2.lines[0].set_xdata(data.dc_voltage[0:(i+1)])
                #ax2.lines[0].set_ydata(data.dc_current[0:(i+1)])
                #ax2.relim()
                #ax2.autoscale()
                

                plt.tight_layout()
                display.clear_output(wait=True)
                display.display(plt.gcf())
    except: 
        print('except'); # do nothing if there was an exception. just proceed to what follows the try-except statement
    
    print(a,b,r_value,p_value,std_err)
    
    display.clear_output(wait=True)
    
    # save data as a .pkl file for use with python
    savedata(full_filename,data)
    
    # also save data as a text file for easy opening with other software

    
    # zero the dc voltage
    #SweepBothGate(k2400,E3640A,0)
    sweep2400(k2400,0)

    # zero the magnetic field
    IsweepBoth(hp6031_U,hp6031_D,0)
    return data





NameError: name 'np' is not defined